In [1]:
import matplotlib.pyplot as plt # For plotting
import numpy as np              # Linear algebra library
import pandas as pd

In [2]:
! pwd
expr_df = pd.read_csv("../metadata/length_and_depth.csv")
expr_df = expr_df.drop("nvar", axis=1)
expr_df = expr_df.rename(columns={
    "length" : "expr_length",
    "depth" : "expr_depth"
})
expr_df

/Users/felixgao/Desktop/Year3/STA378/OptimizationParameterTuning/Julia Notebook/src


,problem,objtype,variable_nvar,expr_length,expr_depth
0,NZF1,least_squares,True,250,8
1,arglina,least_squares,True,10404,9
2,arglinb,least_squares,True,40200,8
3,arglinc,other,True,39204,7
4,argtrig,other,True,497,7
...,...,...,...,...,...
100,tridia,other,True,398,6
101,vardim,other,True,798,6
102,vibrbeam,least_squares,False,481,13
103,watson,least_squares,False,3900,12


In [3]:
complete_df = pd.read_csv("../results/complete_dataset_as_of_nov6.csv")
complete_df = complete_df[complete_df["is_init_run"] == 0]
complete_df = complete_df.rename(columns={"name" : "problem"})
df = complete_df.merge(
    expr_df,
    on = 'problem',
    how = "inner"
)
df

,status,problem,solver,mem,nvar,time,memory,num_iter,nvmops,neval_obj,...,neval_grad,init_eval_grad_time,init_eval_grad_mem,init_eval_grad_alloc,is_init_run,is_scalable,objtype,variable_nvar,expr_length,expr_depth
0,first_order,NZF1,LBFGSSolver,1,91,1.409402,0.125416,287,287,328,...,315,0.000031,0.001440,5,False,False,least_squares,True,250,8
1,first_order,NZF1,LBFGSSolver,2,91,1.407668,0.095296,199,199,223,...,216,0.000019,0.001440,5,False,False,least_squares,True,250,8
2,first_order,NZF1,LBFGSSolver,3,91,1.407882,0.103456,213,213,234,...,228,0.000016,0.001440,5,False,False,least_squares,True,250,8
3,first_order,NZF1,LBFGSSolver,4,91,1.407476,0.098352,191,191,209,...,203,0.000017,0.001440,5,False,False,least_squares,True,250,8
4,first_order,NZF1,LBFGSSolver,5,91,1.407906,0.108416,209,209,231,...,225,0.000017,0.001440,5,False,False,least_squares,True,250,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18821,first_order,woods,LBFGSSolver,96,1000,1.482090,3.197624,64,64,84,...,75,0.001356,0.008664,5,False,True,other,True,425,7
18822,first_order,woods,LBFGSSolver,97,1000,1.481800,3.229800,64,64,84,...,75,0.001373,0.008664,5,False,True,other,True,425,7
18823,first_order,woods,LBFGSSolver,98,1000,1.482000,3.261976,64,64,84,...,75,0.001367,0.008664,5,False,True,other,True,425,7
18824,first_order,woods,LBFGSSolver,99,1000,1.481908,3.294152,64,64,84,...,75,0.001358,0.008664,5,False,True,other,True,425,7


In [4]:
problems = df["problem"].unique()
rng = np.random.default_rng(seed=42)
rng.shuffle(problems)

n = len(problems)
n_train = int(0.7 * n)
n_valid = int(0.15 * n)

train_problems = problems[:n_train]
valid_problems = problems[n_train:n_train + n_valid]
test_problems  = problems[n_train + n_valid:]

train_df = df[df["problem"].isin(train_problems)].reset_index(drop=True)
valid_df = df[df["problem"].isin(valid_problems)].reset_index(drop=True)
test_df  = df[df["problem"].isin(test_problems)].reset_index(drop=True)

In [12]:
from sklearn.ensemble import RandomForestRegressor
feature_cols = ["nvar", 
                "expr_length", 
                "expr_depth", 
                "mem",
                "init_eval_obj_time", 
                "init_eval_obj_mem",
                "init_eval_obj_alloc",
                "init_eval_grad_time",
                "init_eval_grad_mem",
                "init_eval_grad_alloc"]
target_col = "time"
X_train = train_df[feature_cols].to_numpy(dtype=float)
X_valid = valid_df[feature_cols].to_numpy(dtype=float)
X_test  = test_df[feature_cols].to_numpy(dtype=float)

t_train = np.log(train_df[target_col].to_numpy(dtype=float))
t_valid = np.log(valid_df[target_col].to_numpy(dtype=float))
t_test  = np.log(test_df[target_col].to_numpy(dtype=float))

In [13]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

best_rf = None
best_score = np.inf
best_params = None

for n_estimators in [100, 200, 400]:
    for max_depth in [None, 8, 12]:
        for min_leaf in [1, 3, 5]:
            rf = RandomForestRegressor(
                n_estimators=n_estimators,
                max_depth=max_depth,
                min_samples_leaf=min_leaf,
                random_state=0,
                n_jobs=-1,
            )
            rf.fit(X_train, t_train)

            pred_valid = rf.predict(X_valid)
            mse_valid = mean_squared_error(t_valid, pred_valid)

            if mse_valid < best_score:
                best_score = mse_valid
                best_rf = rf
                best_params = (n_estimators, max_depth, min_leaf)

print("Best validation MSE (forest):", best_score)
print("Best params (n_estimators, max_depth, min_leaf):", best_params)


Best validation MSE (forest): 1.2767138217477783
Best params (n_estimators, max_depth, min_leaf): (400, 8, 1)


In [14]:
best_n_estimators, best_max_depth, best_min_leaf = best_params

X_train_full = np.vstack([X_train, X_valid])
t_train_full = np.concatenate([t_train, t_valid])

final_rf = RandomForestRegressor(
    n_estimators=best_n_estimators,
    max_depth=best_max_depth,
    min_samples_leaf=best_min_leaf,
    random_state=0,
    n_jobs=-1,
)
final_rf.fit(X_train_full, t_train_full)

y_pred_test = final_rf.predict(X_test)

test_mse = mean_squared_error(t_test, y_pred_test)
test_mae = mean_absolute_error(t_test, y_pred_test)
test_r2  = r2_score(t_test, y_pred_test)

print("Random forest test MSE:", test_mse)
print("Random forest test MAE:", test_mae)
print("Random forest test R^2:", test_r2)


Random forest test MSE: 3.2733870589931957
Random forest test MAE: 1.138328321211935
Random forest test R^2: 0.009938660447675218


In [15]:
problem_feature_cols = [c for c in feature_cols if c != "mem"]

def choose_best_mem(model, x_problem, mem_candidates):
    preds = []
    for mem in mem_candidates:
        x = np.concatenate([x_problem, [mem]])
        preds.append((mem, model.predict(x.reshape(1, -1))[0]))
    return min(preds, key=lambda x: x[1])


In [16]:
mem_candidates_global = np.sort(train_df["mem"].unique())
problem_feature_cols = [c for c in feature_cols if c != "mem"]

def evaluate_mem_selector(model, df_split):
    rows = []

    for prob, group in df_split.groupby("problem"):
        # restrict candidates to mems that actually appear for this problem
        mems_available = np.sort(group["mem"].unique())
        mem_candidates = [m for m in mem_candidates_global if m in mems_available]
        if len(mem_candidates) == 0:
            # skip weird problems with no mem info
            continue

        # build problem feature vector (no mem)
        row0 = group.iloc[0]
        x_problem = row0[problem_feature_cols].to_numpy(dtype=float)

        # model recommendation
        mem_pred, _ = choose_best_mem(model, x_problem, mem_candidates)

        # true best mem and its time
        best_row = group.loc[group["time"].idxmin()]
        mem_best = best_row["mem"]
        time_best = best_row["time"]

        # time if we use the model's mem; there can be multiple rows with same mem, take min
        time_chosen = group.loc[group["mem"] == mem_pred, "time"].min()

        rows.append(
            {
                "problem": prob,
                "mem_pred": mem_pred,
                "mem_best": mem_best,
                "time_best": time_best,
                "time_chosen": time_chosen,
            }
        )

    res = pd.DataFrame(rows)
    if len(res) == 0:
        print("Warning: no problems evaluated, check your split or column names")
        return res, np.nan, np.nan

    frac_exact = (res["mem_pred"] == res["mem_best"]).mean()
    avg_ratio = (res["time_chosen"] / res["time_best"]).mean()

    return res, frac_exact, avg_ratio

test_res_rf, frac_exact_rf, avg_ratio_rf = evaluate_mem_selector(final_rf, test_df)
print("Forest mem selector exact match fraction:", frac_exact_rf)
print("Forest average time_chosen / time_best:", avg_ratio_rf)



Forest mem selector exact match fraction: 0.0
Forest average time_chosen / time_best: 1.4649070373546162


In [17]:
# 0.1: global best mem on TRAIN
train_mem_stats = (
    train_df.groupby("mem")["time"]
    .mean()
    .sort_values()
)
print(train_mem_stats)

global_best_mem = train_mem_stats.idxmin()
print("Global best mem on train:", global_best_mem)


mem
99    10.283809
72    10.289129
89    10.301934
86    10.302818
55    10.303998
        ...    
5     11.833018
4     12.210107
3     15.978661
2     16.114264
1     20.510161
Name: time, Length: 100, dtype: float64
Global best mem on train: 99


In [18]:
def evaluate_fixed_mem(df_split, fixed_mem):
    rows = []
    for prob, group in df_split.groupby("problem"):
        best_row = group.loc[group["time"].idxmin()]
        mem_best = best_row["mem"]
        time_best = best_row["time"]

        if fixed_mem not in group["mem"].values:
            continue

        time_fixed = group.loc[group["mem"] == fixed_mem, "time"].min()

        rows.append(
            {
                "problem": prob,
                "mem_best": mem_best,
                "time_best": time_best,
                "time_fixed": time_fixed,
            }
        )

    res = pd.DataFrame(rows)
    frac_exact = (res["mem_best"] == fixed_mem).mean()
    avg_ratio = (res["time_fixed"] / res["time_best"]).mean()
    return res, frac_exact, avg_ratio

baseline_res, baseline_frac, baseline_ratio = evaluate_fixed_mem(test_df, global_best_mem)
print("Baseline avg time_fixed / time_best:", baseline_ratio)
print("Forest avg time_chosen / time_best:", avg_ratio_rf)
print("Baseline median ratio:", (baseline_res["time_fixed"] / baseline_res["time_best"]).median())
print("Forest median ratio:", (test_res_rf["time_chosen"] / test_res_rf["time_best"]).median())


Baseline avg time_fixed / time_best: 1.011603149432735
Forest avg time_chosen / time_best: 1.4649070373546162
Baseline median ratio: 1.0004117555900267
Forest median ratio: 1.0038346692445876
